In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install pathlib
%pip install scikit-learn
%pip install imbalanced-learn
%pip install scipy
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np

X_train=np.load("X_train.npy")
X_val=np.load("X_val.npy")
X_test=np.load("X_test.npy")
y_train=np.load("y_train.npy")
y_val=np.load("y_val.npy")
y_test=np.load("y_test.npy")

In [ ]:
import itertools
import pandas as pd

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight


class BusquedaHiperparametros:

    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):

        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test= y_test

        self.mejor_score = -1
        self.mejor_modelo = None
        self.mejores_parametros = None
        self.nombre_modelo= None

        clases_unicas = np.unique(y_train)

        pesos = compute_class_weight(
            class_weight="balanced",
            classes=clases_unicas,
            y=y_train
        )

        #Puedo implementar funciones custom class weight para no usar balanced???

        self.class_weight = dict(zip(clases_unicas, pesos))

        # Para XGBoost 
        self.sample_weight = np.array(
            [self.class_weight[y] for y in y_train]
        )

    def crear_modelo(self, tipo_modelo, parametros):

        if tipo_modelo == "rf":

            from sklearn.ensemble import RandomForestClassifier

            return RandomForestClassifier(
                class_weight=self.class_weight,
                random_state=42,
                n_jobs=-1,
                **parametros
            )

        elif tipo_modelo == "svm":

            from sklearn.svm import SVC

            return SVC(
                class_weight=self.class_weight,
                probability=True,
                **parametros
            )

        elif tipo_modelo == "xgb":

            from xgboost import XGBClassifier

            return XGBClassifier(
                eval_metric="mlogloss",
                random_state=42,
                n_jobs=-1,
                **parametros
            )

        else:
            raise ValueError("Modelo no soportado")


    def obtener_grid(self, tipo_modelo):

        if tipo_modelo == "rf":
            return {
                "n_estimators": [200, 400],
                "max_depth": [None, 20],
                "min_samples_leaf": [1, 4]
            }

        elif tipo_modelo == "svm":
            return {
                "C": [1, 10, 100],
                "gamma": ["scale", 0.01],
                "kernel": ["rbf"]
            }

        elif tipo_modelo == "xgb":
            return {
                "n_estimators": [300, 600],
                "max_depth": [4, 6],
                "learning_rate": [0.05, 0.1]
            }

    def buscar(self, tipo_modelo):

        grid = self.obtener_grid(tipo_modelo)

        claves = grid.keys()
        valores = grid.values()


        for combinacion in itertools.product(*valores):

            parametros = dict(zip(claves, combinacion))

            print(f"Probando {tipo_modelo} con parámetros {parametros}")

            modelo = self.crear_modelo(tipo_modelo, parametros)

            if tipo_modelo == "xgb":

                modelo.fit(
                    self.X_train,
                    self.y_train,
                    sample_weight=self.sample_weight,
                    eval_set=[(self.X_val, self.y_val)],
                    verbose=False
                )

            else:
                modelo.fit(self.X_train, self.y_train)

            predicciones = modelo.predict(self.X_val)

            score = f1_score(self.y_val, predicciones, average="macro")

            print("Modelo validado: ", tipo_modelo)
            print("F1 macro en set de validacion:", score)
            print("Parametros validados:", parametros)


            if score > self.mejor_score:

                self.mejor_score = score
                self.mejor_modelo = modelo
                self.mejores_parametros = parametros
                self.nombre_modelo= tipo_modelo
            
        
    def evaluar_test(self):

        print("Evaluación en Test")
        print("Mejor modelo tras validaciones: ",self.nombre_modelo)
        print("Mejores parametros para dicho modelo: ",self.mejores_parametros)

        pred_test = self.mejor_modelo.predict(self.X_test)

        print(classification_report(self.y_test,pred_test,target_names=self.clases,digits=4))

        print("Accuracy:", accuracy_score(self.y_test, pred_test))
        print("F1 Macro:", f1_score(self.y_test, pred_test, average="macro"))
        print("Recall por clase:", recall_score(self.y_test, pred_test, average=None))

        matrizDeConfusion = confusion_matrix(self.y_test, pred_test)

        print(matrizDeConfusion)

In [11]:
busqueda = BusquedaHiperparametros(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test)

for modelo in ["xgb", "rf", "svm"]:
    busqueda.buscar(modelo)
    
busqueda.evaluar_test()

Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.05}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9451831562140736
Parametros validados: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.05}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9418769823596918
Parametros validados: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.05}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9324256969142993
Parametros validados: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.05}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.1}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9322443684093926
Parametros validados: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.1}
Probando xgb con parámetros 

MemoryError: Unable to allocate 14.6 MiB for an array with shape (1911456, 1) and data type int64